# StyleGAN Human

Made by [Artem Konevskikh](https://aiculedssul.net/)

Based on [StyleGAN-Human](https://github.com/stylegan-human/StyleGAN-Human)

## Setup

In [ ]:
#@title GPU Check
#@markdown You can check which GPU you got. V100 is perfect, P100 and T4 is good, but if you got K80 its better to restart the notebook by selecting **Disconnect and delete runtime** in **Runtime** menu, because it is very slow
!nvidia-smi -L


In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive to load pretrained models and to save the results.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install
!git clone https://github.com/stylegan-human/StyleGAN-Human.git

!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 

!pip install lpips



In [4]:
#@title Load libraries
# %tensorflow_version 1.x 
import os
from PIL import Image
import cv2
import numpy as np
import shlex

In [ ]:
#@title Download models

repo_name='StyleGAN-Human'
os.chdir(f'/content/{repo_name}')

def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../pretrained_models. """
    save_path = os.path.join('/content/', f'{repo_name}',"pretrained_models")
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url

MODEL_PATHS = {
    "stylegan1_1024": {"id": "1h-R-IV-INGdPEzj4P9ml6JTEvihuNgLX", "name": "stylegan1_1024.pkl"},
    "stylegan2_1024": {"id": "1FlAb1rYa0r_--Zj_ML8e6shmaF28hQb5", "name": "stylegan2_1024.pkl"},
    "stylegan2_512": {"id": "1dlFEHbu-WzQWJl7nBBZYcTyo000H9hVm", "name": "stylegan2_512.pkl"},
    "stylegan3_512": {"id": "1_274jk_N6WSCkKWeu7hjHycqGvbuOFf5", "name": "stylegan3_512.pkl"},
}
for model, params in MODEL_PATHS.items():
  download_command = get_download_model_command(file_id=params["id"], file_name=params["name"])
  !{download_command}

ffhq_ckpt = get_download_model_command(file_id="125OG7SMkXI-Kf2aqiwLLHyCvSW-gZk3M", file_name='ffhq.pkl')
!{ffhq_ckpt}
dlib_detector = get_download_model_command(file_id="1MduBgju5KFNrQfDLoQXJ_1_h5MnctCIG", file_name='mmod_human_face_detector.dat')
!{dlib_detector}
dlib_landmark = get_download_model_command(file_id="1A82DnJBJzt8wI2J8ZrCK5fgHcQ2-tcWM", file_name='shape_predictor_68_face_landmarks.dat')
!{dlib_landmark}

## Generate

In [ ]:
#@title Generate full-body images

#@markdown Select model
experiment_type = 'stylegan2_1024' #@param ['stylegan1_1024', 'stylegan2_1024', 'stylegan2_512', 'stylegan3_512']
#@markdown Directory to save the generated images
outdir = "/content/results/1" #@param {type: "string"}
#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Provide one number, range or comma-separated list of integers
seeds = "42,333" #@param {type: "string"}

version=experiment_type.split("_")[0][-1]
outdir = shlex.quote(outdir)
!python generate.py --outdir={outdir} --seeds={seeds} --trunc={truncation} --network=/content/StyleGAN-Human/pretrained_models/{experiment_type}.pkl --version {version}

In [ ]:
#@title Visualize
output_images = [os.path.join(f"{outdir}", x) for x in os.listdir(f"{outdir}")]
for idx, image_path in enumerate(output_images):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if idx == 0:
        res = image
    else:
        res = np.concatenate([res, image], axis=1)

res = Image.fromarray(res)
res

In [ ]:
#@title Interpolation
#@markdown Select model
experiment_type = 'stylegan2_1024' #@param ['stylegan1_1024', 'stylegan2_1024', 'stylegan2_512', 'stylegan3_512']
#@markdown Directory to save the generated images
outdir = "/content/results/2" #@param {type: "string"}
#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Provide one number, range or comma-separated list of integers
seeds = "42,333" #@param {type: "string"}
#@markdown Number of interpolation images
num_interps = 100 #@param {type: "integer"}
#@markdown FPS
fps = 10 #@param {type: "integer"}

outdir = shlex.quote(outdir)
!python interpolation.py --outdir={outdir} --seeds={seeds} --trunc={truncation} --network=/content/StyleGAN-Human/pretrained_models/{experiment_type}.pkl --fps={fps} --num_interps={num_interps}

In [ ]:
#@title Editing
#@markdown Select model
experiment_type = 'stylegan2_1024' #@param ['stylegan1_1024', 'stylegan2_1024', 'stylegan2_512', 'stylegan3_512']
#@markdown Directory to save the generated images
outdir = "/content/results/3" #@param {type: "string"}
#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Provide one number, range or comma-separated list of integers
seeds = "42,420" #@param {type: "string"}
#@markdown Attribute
attr_name = 'bottom_length'  #@param ['upper_length', 'bottom_length']

outdir = shlex.quote(outdir)
!python edit.py --outdir={outdir} --seeds={seeds} --trunc={truncation} --network=/content/StyleGAN-Human/pretrained_models/{experiment_type}.pkl --attr_name {attr_name}

In [ ]:
#@title Style-mixing
#@markdown Select model
experiment_type = 'stylegan2_1024' #@param ['stylegan1_1024', 'stylegan2_1024', 'stylegan2_512', 'stylegan3_512']
#@markdown Directory to save the generated images
outdir = "/content/results/4" #@param {type: "string"}
#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown Random seeds to use for image rows
rows = "42,333" #@param {type: "string"}
#@markdown Random seeds to use for image columns
cols = "23,420" #@param {type: "string"}
#@markdown Style layer range (from 0 to 12?)
styles = "0-6" #@param {type: "string"}

outdir = shlex.quote(outdir)
!python style_mixing.py --outdir={outdir} --trunc={truncation} --network=/content/StyleGAN-Human/pretrained_models/{experiment_type}.pkl --rows={rows} --cols={cols} --styles={styles}

In [ ]:
#@title Style-mixing video
#@markdown Select model
experiment_type = 'stylegan2_1024' #@param ['stylegan1_1024', 'stylegan2_1024', 'stylegan2_512', 'stylegan3_512']
#@markdown Directory to save the generated images
outdir = "/content/results/5" #@param {type: "string"}
#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown Random seeds to use for image rows
row = 42 #@param {type: "integer"}
#@markdown Random seeds to use for image columns
cols = "23,420" #@param {type: "string"}
#@markdown Style layer range (from 0 to 12?)
styles = "0-3" #@param {type: "string"}
#@markdown Duration
duration = 10 #@param {type: "integer"}
#@markdown FPS
fps = 10 #@param {type: "integer"}

outdir = shlex.quote(outdir)
!python stylemixing_video.py --outdir={outdir} --trunc={truncation} --network=/content/StyleGAN-Human/pretrained_models/{experiment_type}.pkl --row-seed={row} --col-seeds={cols} --col-styles={styles} --duration-sec={duration} --fps={fps} 

In [ ]:
#@title InsetGAN (use StyleGAN2-FFHQ as face generator)

#@markdown Select model
experiment_type = 'stylegan2_1024' #@param ['stylegan1_1024', 'stylegan2_1024', 'stylegan2_512', 'stylegan3_512']
#@markdown Directory to save the generated images (should be relative path)
outdir = "../../content/results/6" #@param {type: "string"}
#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown Face seed
face_seed = 9 #@param {type: "integer"}
#@markdown Body seed
body_seed = 333 #@param {type: "integer"}
#@markdown Number of steps for joint optimization
joint_steps = 500 #@param {type: "integer"}
#@markdown Save video
video = True #@param {type: "boolean"}

video = int(video)
outdir = shlex.quote(outdir)
!python insetgan.py --face_seed={face_seed} --body_seed={body_seed} \
                    --joint_steps={joint_steps} --video {video} \
                    --trunc={truncation} --outdir={outdir} \
                    --body_network=/content/StyleGAN-Human/pretrained_models/{experiment_type}.pkl